In [1]:
import pandas as pd
import numpy as np
import re
import streamlit as st

In [2]:
aldi = pd.read_csv("aldi_all_products.csv")
aldi.head()

,name,price
0,"Original Kettle Chips, 8 oz",$1.99
1,"Blueberry Breakfast Biscuits, 5 pack",$3.85
2,"Organic Unrefined Coconut Oil, 14 fl oz",$5.29
3,"Plain Whole Milk Greek Yogurt, 32 oz",$4.15
4,"Parsley Flakes, 0.25 oz",$1.09


In [3]:
#Clean Aldi price column
aldi['price'] = aldi['price'].str.replace("$", "")
aldi['price'] = aldi['price'].astype("float64")

In [4]:
hyvee = pd.read_csv("hyvee_all_products.csv")
hyvee.head()

,name,price
0,"Chocolate Chip Cookies, 12ct",$5.99
1,"Bakery Fresh 12"" Ghirardelli Chocolate Chip Co...",$15.99
2,Sourdough Artisan Bread,$4.99
3,French Loaf 2 Count,$3.49
4,Bakery Fresh Monster Cookies with Ghirardelli ...,$5.99


In [5]:
def clean_price(raw_price):
    if not isinstance(raw_price, str):
        return None

    raw_price = raw_price.strip().lower()

    # Step 1: Convert cents to dollars
    cent_match = re.search(r'(\d+)\s*¢', raw_price)
    if cent_match:
        cents = int(cent_match.group(1))
        return round(cents / 100, 2)

    # Step 2: Convert $X.XX patterns
    dollar_match = re.search(r'\$?(\d+(?:\.\d{1,2})?)', raw_price)
    if dollar_match:
        return float(dollar_match.group(1))

    return None  # fallback if no price found

In [6]:
hyvee['price'] = hyvee['price'].apply(clean_price)

In [7]:
hyvee.head()

,name,price
0,"Chocolate Chip Cookies, 12ct",5.99
1,"Bakery Fresh 12"" Ghirardelli Chocolate Chip Co...",15.99
2,Sourdough Artisan Bread,4.99
3,French Loaf 2 Count,3.49
4,Bakery Fresh Monster Cookies with Ghirardelli ...,5.99


In [8]:
def compare_grocery_prices(
    shopping_list,
    df1,
    df2,
    name_col='name',
    price_col='price',
    store_names=('Store 1', 'Store 2')
):
    total_1 = 0.0
    total_2 = 0.0
    missing_items = []
    comparison_table = []

    print(f"{'Item':<15} | {store_names[0]:<40} | {store_names[1]:<40}")
    print("-" * 110)

    for item in shopping_list:
        # Find matching rows sorted by price
        match_1 = df1[df1[name_col].str.contains(item, case=False, na=False)].sort_values(price_col)
        match_2 = df2[df2[name_col].str.contains(item, case=False, na=False)].sort_values(price_col)

        row_1 = match_1.iloc[0] if not match_1.empty else None
        row_2 = match_2.iloc[0] if not match_2.empty else None

        name_1 = row_1[name_col] if row_1 is not None else "N/A"
        price_1 = row_1[price_col] if row_1 is not None else None

        name_2 = row_2[name_col] if row_2 is not None else "N/A"
        price_2 = row_2[price_col] if row_2 is not None else None

        if price_1 is not None:
            total_1 += price_1
        else:
            missing_items.append((item, store_names[0]))

        if price_2 is not None:
            total_2 += price_2
        else:
            missing_items.append((item, store_names[1]))

        print(f"{item:<15} | {f'{name_1} (${price_1:.2f})' if price_1 else 'N/A':<40} | {f'{name_2} (${price_2:.2f})' if price_2 else 'N/A':<40}")

        comparison_table.append({
            'item': item,
            store_names[0] + '_item': name_1,
            store_names[0] + '_price': price_1,
            store_names[1] + '_item': name_2,
            store_names[1] + '_price': price_2,
        })

    print("\nTOTAL")
    print(f"{store_names[0]}: ${total_1:.2f}")
    print(f"{store_names[1]}: ${total_2:.2f}")

    if missing_items:
        print("\n⚠️ Missing Items:")
        for item, store in missing_items:
            print(f"- {item} not found in {store}")

    """return {
        'total_1': round(total_1, 2),
        'total_2': round(total_2, 2),
        'store_1_name': store_names[0],
        'store_2_name': store_names[1],
        'item_breakdown': comparison_table,
        'missing': missing_items
    }"""
    return None


In [9]:
shopping_list = ["milk", "eggs", "bread", "cheese", "banana"]

In [10]:
compare_grocery_prices(shopping_list, hyvee, aldi, store_names=("Hy-Vee", "Aldi"))

Item            | Hy-Vee                                   | Aldi                                    
--------------------------------------------------------------------------------------------------------------
milk            | So Delicious Dairy Free Coconut Milk Blueberry Yogurt Alternative ($1.88) | Evaporated Milk, 12 fl oz ($1.05)       
eggs            | Hy-Vee Eggs Hard Cooked 6Ct ($2.00)      | Grade A Large White Eggs, 1 dozen ($2.59)
bread           | Hy-Vee White Bread ($1.99)               | Cornbread Stuffing Mix, 6 oz ($1.05)    
cheese          | Hy-Vee Original Macaroni & Cheese Dinner ($0.99) | Macaroni and Cheese, 7.25 oz ($0.65)    
banana          | Fresh Chiquita Bananas ($0.22)           | Bananas, per lb ($0.17)                 

TOTAL
Hy-Vee: $7.08
Aldi: $5.51


In [11]:
st.title("🛒 Grocery Price Comparison")

st.markdown("Enter your grocery list (one item per line):")
input_text = st.text_area("Shopping List", height=200)

if st.button("Compare Prices"):
    if input_text.strip():
        shopping_list = [line.strip() for line in input_text.splitlines() if line.strip()]
        result_df, total_hyvee, total_aldi = compare_grocery_prices(shopping_list, df_hyvee, df_aldi)

        st.markdown("### 🧾 Price Breakdown")
        st.dataframe(result_df, use_container_width=True)

        st.markdown("### 💵 Total Costs")
        col1, col2 = st.columns(2)
        col1.metric("Hy-Vee Total", f"${total_hyvee:.2f}")
        col2.metric("Aldi Total", f"${total_aldi:.2f}")
    else:
        st.warning("Please enter at least one item.")

2025-10-06 18:15:37.926 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-06 18:15:38.473 
  command:

    streamlit run C:\Users\jerri\AppData\Roaming\Python\Python313\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-10-06 18:15:38.473 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-06 18:15:38.473 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-06 18:15:38.474 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-06 18:15:38.474 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-06 18:15:38.474 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-06 18:15:38.476 Thre